<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/JARVIS_QuantumEspressoColab_Basic_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run Quantum espresso calculations with JARVIS-Tools

In [1]:
!pip install jarvis-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 29.9 MB/s eta 0:00:00


In [ ]:
%%time
import os
# Clone the QE git
if not os.path.exists('q-e'):
   os.system('git clone https://github.com/QEF/q-e.git')
# Install the dependencies
!apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc
%cd q-e
# run the configure file
!DFLAGS='-D__OPENMP -D__FFTW3 -D__MPI -D__SCALAPACK' FFT_LIBS='-lfftw3'  ./configure --enable-openmp
! make pw

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libfftw3-bin libfftw3-long3 libfftw3-quad3 libfftw3-single3
The following NEW packages will be installed:
  libfftw3-3 libfftw3-bin libfftw3-dev libfftw3-doc libfftw3-long3
  libfftw3-quad3 libfftw3-single3
0 upgraded, 7 newly installed, 0 to remove and 38 not upgraded.
Need to get 4,251 kB of archives.
After this operation, 23.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libfftw3-long3 amd64 3.3.8-2ubuntu1 [313 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libfftw3-single3 amd64 3.3.8-2ubuntu1 [756 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libfftw3-3 amd64 3.3.8-2ubuntu1 [1,760 B]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libfftw3-quad3 amd64 3.3.8-2ubuntu1 [673 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/main amd64 libfftw3-bin

In [ ]:
#!make ph

In [ ]:
!lscpu 

In [ ]:
from jarvis.core.atoms import Atoms
from jarvis.db.figshare import data
import numpy as np
import pandas as pd

dft_3d = data('dft_3d')
df=pd.DataFrame(dft_3d)
#Filter based on elements
def has_elements(atoms_dict = {}, my_element = ['Al','O']):
    atoms = Atoms.from_dict(atoms_dict)
    elements = atoms.elements
    if set(my_element)==set(elements):
        return True
    else:
        return np.nan
    
df['el'] = df['atoms'].apply(lambda x: has_elements(atoms_dict = x, my_element = ['Si']))
df1 = df.dropna()
print(df_eform_filter = df1[df1['formation_energy_peratom']==0])



In [ ]:
#Lets select JVASP-1002
from jarvis.db.figshare import get_jid_data
from jarvis.tasks.qe.super import SuperCond
from jarvis.core.utils import get_factors
from jarvis.core.atoms import Atoms
from jarvis.db.figshare import data, get_jid_data
from jarvis.core.kpoints import Kpoints3D
from jarvis.tasks.queue_jobs import Queue
from jarvis.db.jsonutils import dumpjson
import os
from jarvis.analysis.structure.spacegroup import Spacegroup3D
import glob
from jarvis.db.jsonutils import loadjson

jid = 'JVASP-1002' #Silicon
dat = get_jid_data(jid=jid, dataset="dft_3d")

In [ ]:
def non_prime_kpoints(kpts=[]):
    """Get non prime kpoints (used for supercon calculations)."""
    mem = []
    for i in kpts:
        facts = get_factors(i)
        if len(facts) == 1:
            val = i + 1
        else:
            val = i
        mem.append(val)
    return mem

qe_cmd = "/content/q-e/bin/pw.x"
a_atoms = Atoms.from_dict(dat["atoms"])
atoms = Spacegroup3D(a_atoms).refined_atoms.get_primitive_atoms
kp = Kpoints3D().automatic_length_mesh(lattice_mat=atoms.lattice_mat, length=dat["kpoint_length_unit"])
kpts = kp._kpoints[0]
kpts=non_prime_kpoints(kpts)
kp=Kpoints3D(kpoints=[kpts])
print ('kpts',kpts)


In [ ]:
from jarvis.tasks.qe.qe import QEjob
relax = {
            "control": {
                # "calculation": "'scf'",
                "calculation":  "'vc-relax'",
                "restart_mode": "'from_scratch'",
                "prefix": "'RELAX'",
                "outdir": "'./'",
                "tstress": ".true.",
                "tprnfor": ".true.",
                "disk_io": "'nowf'",
                "wf_collect": ".true.",
                "pseudo_dir": None,
                "verbosity": "'high'",
                "nstep": 100,
            },
            "system": {
                "ibrav": 0,
                "nat": None,
                "ntyp": None,
                "ecutwfc": 45,
                "ecutrho": 250,
                "q2sigma": 1,
                "ecfixed": 44.5,
                "qcutz": 800,
                "occupations": "'smearing'",
                "degauss": 0.01,
                "lda_plus_u": ".false.",
            },
            "electrons": {
                "diagonalization": "'david'",
                "mixing_mode": "'local-TF'",
                "mixing_beta": 0.3,
                "conv_thr": "1d-9",
            },
            "ions": {"ion_dynamics": "'bfgs'"},
            "cell": {"cell_dynamics": "'bfgs'", "cell_dofree": "'all'"},
        }
qejob_relax = QEjob(
    atoms=a_atoms,
    input_params=relax,
    output_file="relax.out",
    qe_cmd=qe_cmd,
    jobname="relax",
    kpoints=kp,
    input_file="arelax.in",
    url=None,
    psp_dir=None,
    psp_temp_name=None,
)

In [ ]:
%%time
info = qejob_relax.runjob()

In [ ]:
!ls

In [ ]:
info

In [ ]:
from jarvis.io.qe.outputs import DataFileSchema
ds = DataFileSchema('/content/q-e/RELAX.save/data-file-schema.xml')

In [ ]:
print(ds.final_energy) #Hartree, QE usually reports data in Ha

In [ ]:
print(ds.final_energy_breakdown) #eV

In [ ]:
print(ds.final_structure)

In [ ]:
print(ds.indir_gap)

In [ ]:
print(ds.nelec)

In [ ]:
print(ds.efermi)

In [ ]:
#Other data available in xml file
import pprint
pprint.pprint(ds.data)